### DeepSurv on Keras 2.x

In [1]:
# Survival Analysis using Keras
import numpy as np

from keras.optimizers import SGD, RMSprop

from lifelines.utils import concordance_index
from lifelines import CoxPHFitter

from lifelines.datasets import load_rossi
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing

import deepsurv_keras

Using TensorFlow backend.


C:\Users\T.Uemura\Envs\python3.6\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Load Rossi dataset

In [ ]:
rossi_dataset = load_rossi()
E = np.array(rossi_dataset["arrest"])
Y = np.array(rossi_dataset["week"])
X = np.array(rossi_dataset)
X = X.astype('float64')
X = X[:, 2:]

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.25, random_state=0)
X_train, X_val, E_train, E_val = train_test_split(X, E, test_size=0.25, random_state=0)

# Standardize
scaler = preprocessing.StandardScaler().fit(X_train[:, [1, 6]])
X_train[:, [1, 6]] = scaler.transform(X_train[:, [1, 6]])
X_val[:, [1, 6]] = scaler.transform(X_val[:, [1, 6]])

# Sorting for NNL!
sort_idx = np.argsort(Y_train)[::-1]
X_train = X_train[sort_idx]
Y_train = Y_train[sort_idx]
E_train = E_train[sort_idx]

### Build and train DeepSurv on Keras 2.x

In [ ]:
model = deepsurv_keras.build_model()

sgd = SGD(lr=1e-5, decay=0.01, momentum=0.9, nesterov=True)
rmsprop = RMSprop(lr=1e-5, rho=0.9, epsilon=1e-8)
model.compile(loss=negative_log_likelihood(E_train), optimizer=sgd)

In [ ]:
print('Training...')
model.fit(X_train, Y_train, batch_size=324, epochs=1000, shuffle=False)  # Shuffle False --> Important!!

### Testing

In [ ]:
hr_pred = model.predict(X_train)
hr_pred = np.exp(hr_pred)
ci = concordance_index(Y_train,-hr_pred,E_train)

hr_pred2 = model.predict(X_val)
hr_pred2 = np.exp(hr_pred2)
ci2 = concordance_index(Y_val,-hr_pred2,E_val)
print('Concordance Index for training dataset:', ci)
print('Concordance Index for test dataset:', ci2)

# Cox Fitting
cf = CoxPHFitter()
cf.fit(rossi_dataset, 'week', event_col='arrest')

cf.print_summary()  # access the results using cf.summary